In [1]:
#!pip install statsmodels
import pandas as pd
import numpy as np

In [2]:
#convert stata data to pandas
stata_data='../NSDUH_2017.DTA'
df=pd.read_stata(stata_data)
t=df.shape[0]

In [3]:
dfa=df[df['CATAG2']!='1 - 12-17 Years Old']
dfa = dfa[df.suicplan != '85 - BAD DATA Logically assigned']
dfa = dfa[df.suicplan != '94 - DON T KNOW']
dfa = dfa[df.suicplan != '97 - REFUSED']
dfa = dfa[df.suicplan != '98 - BLANK']

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [4]:
dfa.groupby(['suicplan']).size()

suicplan
1 - Yes                               845
2 - No                               1740
85 - BAD DATA Logically assigned        0
94 - DON T KNOW                         0
97 - REFUSED                            0
98 - BLANK                              0
99 - LEGITIMATE SKIP                39652
dtype: int64

In [5]:
dfa.loc[df.suicplan=='1 - Yes', 'suicide'] = 'plan'
dfa.loc[df.suicplan=='2 - No', 'suicide'] = 'thought'
dfa.loc[df.suicplan=='99 - LEGITIMATE SKIP', 'suicide'] = 'no'

In [6]:
dfa.groupby('irpnrnmrec').size()

irpnrnmrec
1 - Within the past 30 days                               648
2 - More than 30 days ago but within the past 12 mos     1596
3 - More than 12 months ago                              2953
9 - NEVER MISUSED PAIN RELIEVERS                        37040
dtype: int64

In [7]:
dfa.loc[df.irpnrnmrec=='1 - Within the past 30 days', 'misuse'] = 'last year'
dfa.loc[df.irpnrnmrec=='2 - More than 30 days ago but within the past 12 mos', 'misuse'] = 'last year'
dfa.loc[df.irpnrnmrec=='3 - More than 12 months ago', 'misuse'] = 'sometime'
dfa.loc[df.irpnrnmrec=='9 - NEVER MISUSED PAIN RELIEVERS', 'misuse'] = 'never'

In [8]:
gf=dfa.groupby(['suicide','misuse']).size()

In [9]:
gf.unstack()[['never','sometime','last year']]

misuse,never,sometime,last year
suicide,,,
no,35182,2642,1828
plan,563,104,178
thought,1295,207,238


In [10]:
import statsmodels.api as sm
logit_model=sm.Logit(y,X)
result=logit_model.fit()
print(result.summary2())

ModuleNotFoundError: No module named 'statsmodels'